## Openpose tutorial notebook: VIDEO pose

This notebook provides an introduction to [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose), a tool for detecting human body, hand, facial, and foot keypoints (in total 135 keypoints) from images/videos.

Pros:
* detects 25 body keypoints, 21 hand key points, and 70 face key points from images and videos.
* It is free.
* It can represent human activity in a video through body keypoints, wihout revealing individually identifiable information.
* Keypoints detected from video using OpenPose can be used for applications related to sports, performance, behavior, activity, etc.

Cons:
* Requires a single person to be present to be consistent across frames. If multiple persons are present, it is best to divide the screen to 
separate them (which we use and will show here)

## Step 1: Install OpenPose

**This takes about 15 minutes.**  
If you are on google colab, it can take up to ~40 minutes to install the entire package.

In [6]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
    !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
    !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
    # clone openpose
    !git clone -q --depth 1 $git_repo_url
    !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
    # install system dependencies
    !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
    # install python dependencies
    !pip install -q youtube-dl
    # build openpose
    !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threa

If the above cell executed properly, you should have cmake (essential for running openpose) and `openpose` appear in your directory structure.

## Additional dependencies for openpose

This may take ~5 minutes to run.

In [10]:
# You may need to run this each time you start a new session
# Basic
! sudo dpkg --configure -a
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://deb.nodesource.com/node_16.x focal InRelease                     
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease                 
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease           
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]      
Fetched 222 kB in 1s (237 kB/s)                                     
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.8ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 127 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopencv

## Step 1.1: Install other libraries

In [11]:
# processing libraries

import os 
import json
import pandas as pd
import numpy as np


For this example, we will use conversation data collected to explore how multimodal alignment during conversation is affected by communication medium 
and conversation. 

A subset of our data is collected in remote conversation collected over Zoom and laboratory setting, which we will use for this tutorial.

## Step 2: Specify path settings and preprocessing

for this tutorial we will set a base path where all our data folders and notebooks will be stored

In [27]:
BASE_PATH = os.getcwd()
#print(BASE_PATH)  # this is our current directory

**Now upload/add your data to a folder inside your project directory.**

Here we have used a folder named `conversation_data` with one conversation from remote setting (ZR = Zoom Remote) and the other from laboratory setting (ZT = Zoom Torso) 

In [28]:
ZR_path = os.path.join(BASE_PATH,'conversation_data/ZR')
ZT_path = os.path.join(BASE_PATH,'conversation_data/ZT')

In [29]:
#specify path for a specific video
current_file = '1058_ZT_4_Aff_Video_right.mp4'
video = os.path.join(ZT_path, current_file) 

#print(video)    # path of the video currently being analyzed, remove hash symbol to run the command

In [30]:
# create a folder to store saved video frames with pose rendered
# create a folder to store saved body-point co-ordinates 

# if the folder already exists, will return that confirmation as `File exists`
!cd openpose && mkdir output_jsons
!cd openpose && mkdir "$BASE_PATH"/openpose_videos


mkdir: cannot create directory ‘output_jsons’: File exists
mkdir: cannot create directory ‘/notebooks/openpose_videos’: File exists


Since our videos are about ~10 minutes long, we will create a short segment from the video that contains the first 60 seconds using the following command 
and save it as `sample.mp4` in your main directory.
We will use this video for the rest of the tutorial.

In [19]:
# saving a short segment of the first 60 seconds of our video
!echo y | ffmpeg -ss 0 -i "$video" -c copy -t 60  /notebooks/sample.mp4

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

## Step 3: Run OpenPose and save results

In [21]:
! cd openpose && ./build/examples/openpose/openpose.bin --video "$BASE_PATH"/sample.mp4 --display 0  --write_video "$BASE_PATH"/openpose_videos/sample.mp4 --write_json output_jsons/sample/

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-l

For a 1 minute-long video it takes about 2 minutes to run this.

Once this process finish, we get two types of output:

1. inside `openpose/output_jsons`, we get a folder containing json files with body key point co-ordinates for each frame of the video. 
1. inside `output_videos`, we get a folder containing a video file with body key point co-ordinates rendered on each frame of the video. 

## Step 4: Preparing the data for analysis

OpenPose returns the body keypoints in .json format, where each json file contains information about a specific frame.
How will we use this for analysis?

We can combine all the json files and combine them into a single .csv file per video for analysis.

In [32]:
#create a directory to save the csv files
try:
    os.mkdir(BASE_PATH + '/openpose_csvs')
             
except:
    pass


#specify jsonfile path
json_path = '/notebooks/openpose/output_jsons/sample'
# get all .json files
get_jsons = sorted([x for x in os.listdir(json_path) if x.endswith(".json")])

body_points = []
for json_file in get_jsons:
        
    df = pd.read_json(json_path + '/' + json_file)
            
    try: 
        pose = df.iloc[0]['people']['pose_keypoints_2d']
        
        #getting x, y coordinate from original openpose format (x, y, confidence)
        x = pose[0::3]  # jump by three, take only first element, x
        y = pose[1::3]  # jump by three, take only second element, y
    
        #x, y
        features_x = []
        features_y = []
            
        for i, j in zip(x, y):
            features_x.append(i)
            features_y.append(j)

        features = [[i, j] for (i,j) in zip(features_x, features_y)]

        # x, y from 25 keypoints = 50 values
        features = np.reshape(features, (1, 50))

        feature_columns = ["Nose_x", "Nose_y",  "Neck_x", "Neck_y",  "RShoulder_x", "RShoulder_y", "RElbow_x", "RElbow_y", "RWrist_x", "RWrist_y", "LShoulder_x", "LShoulder_y", "LElbow_x", "LElbow_y", "LWrist_x", "LWrist_y", "MidHip_x", "MidHip_y", "RHip_x", "RHip_y", "RKnee_x", "RKnee_y", "RAnkle_x", "RAnkle_y", "LHip_x", "LHip_y",  "LKnee_x", "LKnee_y", "LAnkle_x",  "LAnkle_y", "REye_x",  "REye_y", "LEye_x",  "LEye_y", "REar_x", "REar_y" ,"LEar_x", "LEar_y", "LBigToe_x", "LBigToe_y", "LSmallToe_x", "LSmallToe_y", "LHeel_x", "LHeel_y", "RBigToe_x", "RBigToe_y", "RSmallToe_x", "RSmallToe_y", "RHeel_x", "RHeel_y"]
    
        data = pd.DataFrame(features, index= [0], columns=feature_columns)
        
        
    #in case there is no person present in a frame
    except:
        pass
        
    
    #stack all frames' information
    try:
        body_points.append(data)
    
    except:
        pass
            
    
    
#saving the results as a csv
try:
        
    body_df = pd.concat(body_points, axis=0, ignore_index = 1)
        
    body_df.to_csv(BASE_PATH + '/openpose_csvs/sample.csv')
    
except:
    print('Could not process:', current_file)
    pass

#let's check the csv file
df = pd.read_csv(BASE_PATH + '/openpose_csvs/sample.csv', index_col = [0])
df

,Nose_x,Nose_y,Neck_x,Neck_y,RShoulder_x,RShoulder_y,RElbow_x,RElbow_y,RWrist_x,RWrist_y,...,LSmallToe_x,LSmallToe_y,LHeel_x,LHeel_y,RBigToe_x,RBigToe_y,RSmallToe_x,RSmallToe_y,RHeel_x,RHeel_y
0,325.819,279.673,331.907,343.828,279.685,343.959,251.557,424.341,249.641,490.590,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,325.819,279.673,331.907,343.828,279.685,343.959,251.557,424.341,249.641,490.590,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,325.819,279.673,331.907,343.828,279.685,343.959,251.557,424.341,249.641,490.590,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,325.819,279.673,331.907,343.828,279.685,343.959,251.557,424.341,249.641,490.590,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,325.819,279.673,331.907,343.828,279.685,343.959,251.557,424.341,249.641,490.590,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,325.827,279.690,331.908,343.822,279.688,343.957,251.559,424.341,249.642,490.584,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1197,325.813,279.684,331.904,343.827,279.681,343.964,251.558,424.341,249.641,490.586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1198,325.812,279.681,331.904,343.826,279.683,343.961,251.559,424.341,249.640,490.588,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1199,325.812,279.679,331.907,343.827,279.684,343.961,251.558,424.341,249.641,490.587,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 5: Putting it all together and running this process in batch on multiple videos

### Defining the entire process as a function

In [33]:
def run_openpose(BASE_PATH, INPUT_VIDEOS, OUTPUT_JSONS, OUTPUT_VIDEOS, OUTPUT_CSVS):
    
    input_videos = sorted([i for i in os.listdir(INPUT_VIDEOS) if not i.startswith('.')])
    
    #create folders in output jsons
    for file in input_videos: 
        try:
            os.mkdir(OUTPUT_JSONS+'/' + file.split('.')[0])
            os.mkdir(OUTPUT_VIDEOS)
            
        except:
            pass
        
    
    #run openpose 
    for file in input_videos:
        
        #get only the name
        file_name = file.split('.')[0]
        print('File ' + file + ' is processing!')
        
        #get a small 2-minute long video, comment this line for processsing full video
        #!echo y | ffmpeg -ss 0 -i "$file" -c copy -t 120  "$INPUT_VIDEOS"/"$file_name"
        !cd openpose && ./build/examples/openpose/openpose.bin --video "$INPUT_VIDEOS"/"$file_name".mp4 --display 0 --render_pose 0 --write_json "$OUTPUT_JSONS"/"$file_name"/
        
        #process json files
        json_path = OUTPUT_JSONS + '/' + file_name
        
        # get all .json files
        get_jsons = sorted([x for x in os.listdir(json_path) if x.endswith(".json")])
        
        body_points = []
        for json_file in get_jsons:
        
            df = pd.read_json(json_path + '/' + json_file)
        
            try: 
                pose = df.iloc[0]['people']['pose_keypoints_2d']
        
                #getting x, y coordinate from original openpose format (x, y, confidence)
                x = pose[0::3]  # jump by three, take only first element, x
                y = pose[1::3]  # jump by three, take only second element, y
    
                #x, y
                features_x = []
                features_y = []
            
                for i, j in zip(x, y):
                    features_x.append(i)
                    features_y.append(j)

                features = [[i, j] for (i,j) in zip(features_x, features_y)]

                # x, y from 25 keypoints = 50 values
                features = np.reshape(features, (1, 50))

                feature_columns = ["Nose_x", "Nose_y",  "Neck_x", "Neck_y",  "RShoulder_x", "RShoulder_y", "RElbow_x", "RElbow_y", "RWrist_x", "RWrist_y", "LShoulder_x", "LShoulder_y", "LElbow_x", "LElbow_y", "LWrist_x", "LWrist_y", "MidHip_x", "MidHip_y", "RHip_x", "RHip_y", "RKnee_x", "RKnee_y", "RAnkle_x", "RAnkle_y", "LHip_x", "LHip_y",  "LKnee_x", "LKnee_y", "LAnkle_x",  "LAnkle_y", "REye_x",  "REye_y", "LEye_x",  "LEye_y", "REar_x", "REar_y" ,"LEar_x", "LEar_y", "LBigToe_x", "LBigToe_y", "LSmallToe_x", "LSmallToe_y", "LHeel_x", "LHeel_y", "RBigToe_x", "RBigToe_y", "RSmallToe_x", "RSmallToe_y", "RHeel_x", "RHeel_y"]
    
                data = pd.DataFrame(features, index= [0], columns=feature_columns)
        
        
            #in case there is no person present in a frame
            except:
                #data = pd.DataFrame(0, columns=feature_columns)
                pass
            
            try:
                body_points.append(data)
    
            except:
                pass
            
    
        #saving the results as a csv
        try:
        
            body_df = pd.concat(body_points, axis=0, ignore_index = 1)
        
            body_df.to_csv(OUTPUT_CSVS + '/' + file_name + '.csv')
    
        except:
            print('Could not process:', file)
            pass
    
    
    return

        
        
        

### Specify path settings

In [34]:
BASE_PATH = os.getcwd()

# where our inpout videos are stored
INPUT_VIDEOS = os.path.join(BASE_PATH,'conversation_data/ZT') 

# path to output jsons and video files
OUTPUT_JSONS = os.path.join(BASE_PATH,'openpose/output_jsons')
OUTPUT_VIDEOS = os.path.join(BASE_PATH,'output_videos')

# path to csv file for analysis
OUTPUT_CSVS = os.path.join(BASE_PATH,'openpose_csvs')

### Run function

In [ ]:
run_openpose(BASE_PATH, INPUT_VIDEOS, OUTPUT_JSONS, OUTPUT_VIDEOS, OUTPUT_CSVS)

File 1058_ZT_4_Aff_Video_left.mp4 is processing!
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 953.602299 seconds.
File 1058_ZT_4_Aff_Video_right.mp4 is processing!
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 718.979986 seconds.
File 1082_ZT_4_Aff_Video_Trim_left.mp4 is processing!
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.


**Congratulations** You now have frame-by-frame pose information as a time series for your video stored in `openpose_csvs` folder.

## We can use different analysis tool such as [PyRQA](https://pypi.org/project/PyRQA/) to analyze this time series.